In [1]:
import matplotlib.pyplot as plt
import os, sys, time
stdin, stdout, stderr = sys.stdin, sys.stdout, sys.stderr
import tensorflow as tf
from model import file_input_fn, lm_model_fn
from utils import data_utils
sys.stdin, sys.stdout, sys.stderr = stdin, stdout, stderr
from os.path import expanduser
HOME = expanduser("~")

Building prefix dict from /usr/local/lib/python2.7/dist-packages/synonyms/data/vocab.txt ...
Loading model from cache /tmp/jieba.u4ce22e8452a5a628ef9a1ba508b45102.cache
Loading model cost 0.685 seconds.
Prefix dict has been built succesfully.


In [2]:
char_vocab_file = HOME+"/Data/Vocab/vocab_zh"
word_vocab_file = HOME+"/Data/Vocab/word_vocab_zh"
char_embedding_files = HOME+"/Data/Vocab/zh_char_300_nlpcc.txt"
data_dir = HOME+"/Data/text_zh"
train_dir = 'estimator_zh_small'

char_vocab = data_utils.Vocab(
    char_vocab_file,
    embedding_files=char_embedding_files)
char_vocab_size = char_vocab.size()

num_steps = 800000
eval_every = 10000

params = {
    'max_lr': 5e-5,
    'num_steps': num_steps,
    'pct_start': 0.3,
    'dropout': 0.1,
    'wd': 1e-7,
    'char_vocab_size': char_vocab_size,
    'char_vocab_dim': 300,
    'char_vocab_emb': char_vocab.embedding_init,
    'max_char_length': 8,
    'layer_size': 256,
    'num_layers': 3,
    'data_spec': [{'type': 'seq', 'is_target': True, 'select_from': None}]
}


batch_size = 32
num_pieces = 1
max_lengths = [256]

In [3]:
gpu_id = '2'
# strategy = tf.distribute.MirroredStrategy(devices=['GPU:2','GPU:3'])
session_config = tf.ConfigProto(allow_soft_placement=True, gpu_options=tf.GPUOptions(visible_device_list=gpu_id))
config=tf.estimator.RunConfig(
#     train_distribute=strategy,
#     eval_distribute=strategy,
    session_config=session_config,
    log_step_count_steps=1000)
lm = tf.estimator.Estimator(
    model_fn=lm_model_fn,
    model_dir=train_dir,
    params=params,
    config=config)

INFO:tensorflow:Using config: {'_save_checkpoints_secs': 600, '_session_config': gpu_options {
  visible_device_list: "2"
}
allow_soft_placement: true
, '_keep_checkpoint_max': 5, '_task_type': 'worker', '_train_distribute': None, '_is_chief': True, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f2b7a33cc10>, '_model_dir': 'estimator_zh_small', '_protocol': None, '_save_checkpoints_steps': None, '_keep_checkpoint_every_n_hours': 10000, '_service': None, '_num_ps_replicas': 0, '_tf_random_seed': None, '_save_summary_steps': 100, '_device_fn': None, '_experimental_distribute': None, '_num_worker_replicas': 1, '_task_id': 0, '_log_step_count_steps': 1000, '_evaluation_master': '', '_eval_distribute': None, '_global_id_in_cluster': 0, '_master': ''}


I0306 20:45:27.533679 139830806021888 estimator.py:201] Using config: {'_save_checkpoints_secs': 600, '_session_config': gpu_options {
  visible_device_list: "2"
}
allow_soft_placement: true
, '_keep_checkpoint_max': 5, '_task_type': 'worker', '_train_distribute': None, '_is_chief': True, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f2b7a33cc10>, '_model_dir': 'estimator_zh_small', '_protocol': None, '_save_checkpoints_steps': None, '_keep_checkpoint_every_n_hours': 10000, '_service': None, '_num_ps_replicas': 0, '_tf_random_seed': None, '_save_summary_steps': 100, '_device_fn': None, '_experimental_distribute': None, '_num_worker_replicas': 1, '_task_id': 0, '_log_step_count_steps': 1000, '_evaluation_master': '', '_eval_distribute': None, '_global_id_in_cluster': 0, '_master': ''}


In [ ]:
for _ in range(int(num_steps / eval_every)):
    lm.train(
        input_fn=lambda: file_input_fn(
            char_vocab, os.path.join(data_dir, 'train'),
            num_pieces, batch_size, max_lengths, True),
        steps=eval_every)
    lm.evaluate(
        input_fn=lambda: file_input_fn(
            char_vocab, os.path.join(data_dir, 'dev'),
            num_pieces, batch_size, max_lengths, False))

Instructions for updating:
Colocations handled automatically by placer.


W0306 20:45:27.590737 139830806021888 deprecation.py:323] From /usr/local/lib/python2.7/dist-packages/tensorflow/python/framework/op_def_library.py:263: colocate_with (from tensorflow.python.framework.ops) is deprecated and will be removed in a future version.
Instructions for updating:
Colocations handled automatically by placer.


Instructions for updating:
tf.py_func is deprecated in TF V2. Instead, use
    tf.py_function, which takes a python function which manipulates tf eager
    tensors instead of numpy arrays. It's easy to convert a tf eager tensor to
    an ndarray (just call tensor.numpy()) but having access to eager tensors
    means `tf.py_function`s can use accelerators such as GPUs as well as
    being differentiable using a gradient tape.
    


W0306 20:45:27.611499 139830806021888 deprecation.py:323] From model.py:68: py_func (from tensorflow.python.ops.script_ops) is deprecated and will be removed in a future version.
Instructions for updating:
tf.py_func is deprecated in TF V2. Instead, use
    tf.py_function, which takes a python function which manipulates tf eager
    tensors instead of numpy arrays. It's easy to convert a tf eager tensor to
    an ndarray (just call tensor.numpy()) but having access to eager tensors
    means `tf.py_function`s can use accelerators such as GPUs as well as
    being differentiable using a gradient tape.
    


INFO:tensorflow:Calling model_fn.


...

W0306 20:45:53.265678 139830806021888 deprecation.py:323] From /usr/local/lib/python2.7/dist-packages/tensorflow/python/training/saver.py:1070: get_checkpoint_mtimes (from tensorflow.python.training.checkpoint_management) is deprecated and will be removed in a future version.
Instructions for updating:
Use standard file utilities to get mtimes.


INFO:tensorflow:Running local_init_op.


I0306 20:45:53.977049 139830806021888 session_manager.py:491] Running local_init_op.


INFO:tensorflow:Done running local_init_op.


I0306 20:45:54.254179 139830806021888 session_manager.py:493] Done running local_init_op.
